# sklearn 문서 전처리

# BOW

BOW(Bag of Words) 인코딩 방법 : 문서를 숫자 벡터로 변환하는 가장 기본적인 방법, BOW 인코딩 방법은 전체 문서 $[d_1, d_2, ..., d_n]$을 구성하는 고정된 단어장(vocabulary) $[t_1, t_2, ..., t_n]$을 만들고 해당 단어들이 포함되어 있는지를 표현

표현 방법 1

$x_{i,j} = 문서 d_i 내의 단어 t_i의 출현 빈도$

표현 방법 2

$\begin{split} x_{i,j} = 
\begin{cases}
0, & \text{만약 단어 $t_j$가 문서 $d_i$ 안에 없으면} \\
1. & \text{만약 단어 $t_j$가 문서 $d_i$ 안에 있으면}
\end{cases}
\end{split}$

# 문서 전처리용 클래스의 종류

**DictVectorizer**:

각 단어의 수를 세어놓은 사전에서 BOW 인코딩 벡터를 만든다.

**CountVectorizer**:

문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩 벡터를 만든다.

**TfidfVectorizer**:

CountVectorizer와 비슷하지만 TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩 벡터를 만든다.

**HashingVectorizer**:

해시 함수(hash function)을 사용하여 적은 메모리와 빠른 속도로 BOW 인코딩 벡터를 만든다.

## DictVectorizer

DictVectorizer는 문서에서 단어의 사용 빈도를 나타내는 딕셔너리 정보를 입력받아 BOW를 생성한다.

**DictVectorizer를 사용하기 위해선 단어의 출현빈도 정보를 가지고 있는 딕셔너리가 준비되어야한다.**

In [3]:
from sklearn.feature_extraction import DictVectorizer

v = DictVectorizer(sparse=False)
    # 1번 문서
D = [{'Apple':1, 'Banana':2}, 
    # 2번 문서
     {'Banana':3, 'Melon':1}]

X = v.fit_transform(D)
X

array([[1., 2., 0.],
       [0., 3., 1.]])

최종적으로 X행렬은 각 단어 Apple, Banana, Melon의 출현 빈도를 의미한다.

DictVectorizer 객체는 feature_names_ 속성에 fit된 단어들에 대한 정보를 담고있다.

In [4]:
# feature_names_ 속성

v.feature_names_

['Apple', 'Banana', 'Melon']

추가적으로 fit 에 사용하지 않은 단어가 추가적으로 입력되면 아예 없는 것으로 처리되기 때문에 되도록이면 vectorizer 객체에 단어들을 fit할 때 가능한 모든 단어를 사용하는 것이 좋다.

In [5]:
v.transform({'Melon':4, 'Orange':10})

array([[0., 0., 4.]])

실제로 transform 메소드로 Melon과 Orange 정보를 넘기면 Orange에 대한 벡터화는 이루어지지 않는 것을 확인할 수 있다.

## CountVectorizer

CountVectorizer는 DictVectorizer에 fit하기 위한 단어의 출현빈도 정보를 가진 딕셔너리를 생성하는 기능을 제공한다. 

1. 문서를 토큰 리스트로 변환
2. 각 문서의 토큰 출현빈도 카운팅
3. 각 문서를 BOW 인코딩 벡터로 변환

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',
]

vect = CountVectorizer()
vect.fit(corpus)

# 단어와 index 번호 확인
vect.vocabulary_

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [9]:
vect.transform(['This is the second document.']).toarray()

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]], dtype=int64)

In [10]:
vect.transform(['Something completely new.']).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [11]:
vect.transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

**CountVectorizer에서 사용 가능한 파라미터 정보**

stop_words : 문자열 {‘english’}, 리스트 또는 None (디폴트)

stop words 목록.‘english’이면 영어용 스탑 워드 사용.

analyzer : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수

단어 n-그램, 문자 n-그램, 단어 내의 문자 n-그램

token_pattern : string

토큰 정의용 정규 표현식

tokenizer : 함수 또는 None (디폴트)

토큰 생성 함수 .

ngram_range : (min_n, max_n) 튜플

n-그램 범위

max_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1

단어장에 포함되기 위한 최대 빈도

min_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1

단어장에 포함되기 위한 최소 빈도

# Stop Words

Stop Words는 문서에서 단어장을 생성할 때 무시할 단어이다. (불용어)

일반적으로 영어의 관사, 접속사, 한국어의 조사 등이 해당한다.

In [12]:
vect = CountVectorizer(stop_words=['and', 'is', 'the', 'this']).fit(corpus)
vect.vocabulary_

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}

In [13]:
vect = CountVectorizer(stop_words="english").fit(corpus)
vect.vocabulary_

{'document': 0, 'second': 1}

# 토큰

analyzer, tokenizer, token_pattern 파라미터로 사용할 토큰 생성기를 선택

In [15]:
# 문자로 토큰화

vect = CountVectorizer(analyzer='char').fit(corpus)
vect.vocabulary_

{'t': 16,
 'h': 8,
 'i': 9,
 's': 15,
 ' ': 0,
 'e': 6,
 'f': 7,
 'r': 14,
 'd': 5,
 'o': 13,
 'c': 4,
 'u': 17,
 'm': 11,
 'n': 12,
 '.': 1,
 'a': 3,
 '?': 2,
 'l': 10}

In [16]:
# 정규표현식 이용
# t로 시작하고 알파벳인 단어만 토큰으로 사용

vect = CountVectorizer(token_pattern="t\w+").fit(corpus)
vect.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

In [18]:
# 외부의 nltk, konlpy등의 좋은 토크나이저를 사용하고 싶은 경우 
# 함수 자체를 인수로 넣어도 무방하다.

import nltk

vect = CountVectorizer(tokenizer=nltk.word_tokenize).fit(corpus)
vect.vocabulary_

{'this': 11,
 'is': 5,
 'the': 9,
 'first': 4,
 'document': 3,
 '.': 0,
 'second': 8,
 'and': 2,
 'third': 10,
 'one': 7,
 '?': 1,
 'last': 6}

# N그램

N그램은 단어장 생성에 사용할 토큰의 크기를 결정.

- 모노그램(monogram) : 토큰 하나만 단어로 사용
- 바이그램(bigram) : 두 개의 연결된 토큰을 하나의 단어로 사용

In [20]:
# 두개의 단어를 하나의 토큰으로 사용

vect = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
vect.vocabulary_

{'this is': 12,
 'is the': 2,
 'the first': 7,
 'first document': 1,
 'the second': 9,
 'second second': 6,
 'second document': 5,
 'and the': 0,
 'the third': 10,
 'third one': 11,
 'is this': 3,
 'this the': 13,
 'the last': 8,
 'last document': 4}

In [22]:
# ngram_range(1, 2)는 모노그램, 바이그램 둘 다 사용하겠다.

vect = CountVectorizer(ngram_range=(1, 2), token_pattern="t\w+").fit(corpus)
vect.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

# 빈도수

- `max_df` : 지정한 값보다 큰 빈도로 사용된 단어는 불용어처리
- `min_df` : 지정한 값보다 작은 빈도로 사용된 단어는 불용어처리

In [23]:
vect = CountVectorizer(max_df=4, min_df=2).fit(corpus)
vect.vocabulary_, vect.stop_words_

({'this': 3, 'is': 2, 'first': 1, 'document': 0},
 {'and', 'last', 'one', 'second', 'the', 'third'})

In [24]:
vect = CountVectorizer(ngram_range=(1, 2), token_pattern="t\w+").fit(corpus)
vect.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

# TF-IDF

TF-IDF(Term Frequency - Inverse Document) 인코딩 방법은 BOW 방법과는 다르게 모든 문서에 대해 공통적인 단어는 문서 구별 능력이 떨어진다고 보며 가중치를 축소하는 인코딩 방식이다.

예를 들어 관사와 같이 a, an, the와 같은 단어는 모든 문서에서 공통적으로 발견되기 때문에 오히려 문서 구별 능력이 떨어진다고 보는 것이다.

책마다 TF-IDF를 계산하는 방식은 약간씩 다르게 소개하나 sklearn에서는 다음의 방법으로 계산한다.

$$\text{tf-idf}(d, t) = \text{tf}(d, t) \cdot \text{idf}(t)$$

idf는 df(document frequency)의 역수로 이에 log를 취해 스케일링한 값이 된다.

즉, 문서 출현 빈도가 높을수록 idf값은 작아지게 된다.

$$\text{idf}(d, t) = \log \dfrac{n}{1 + \text{df}(t)}$$

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidv = TfidfVectorizer().fit(corpus)
tfidv.transform(corpus).toarray()

array([[0.        , 0.38947624, 0.55775063, 0.4629834 , 0.        ,
        0.        , 0.        , 0.32941651, 0.        , 0.4629834 ],
       [0.        , 0.24151532, 0.        , 0.28709733, 0.        ,
        0.        , 0.85737594, 0.20427211, 0.        , 0.28709733],
       [0.55666851, 0.        , 0.        , 0.        , 0.        ,
        0.55666851, 0.        , 0.26525553, 0.55666851, 0.        ],
       [0.        , 0.38947624, 0.55775063, 0.4629834 , 0.        ,
        0.        , 0.        , 0.32941651, 0.        , 0.4629834 ],
       [0.        , 0.45333103, 0.        , 0.        , 0.80465933,
        0.        , 0.        , 0.38342448, 0.        , 0.        ]])

# Hashing Trick

CountVectorizer의 경우 메모리상에서 작업을 수행하기에 처리할 문서의 크기가 커지면 컴퓨터 자원에 부담이 되기 마련이다.

이 때 HashingVectorizer를 사용하면 해시함수를 통해 각 단어에 대한 index 번호를 생성해 메모리 및 실행시간이 단축되는 장점이 있다.

In [28]:
from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups()
len(twenty.data)

11314

In [29]:
%time CountVectorizer().fit(twenty.data).transform(twenty.data)

Wall time: 4.99 s


<11314x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [30]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=300000)

In [31]:
%time hv.transform(twenty.data)

Wall time: 2.02 s


<11314x300000 sparse matrix of type '<class 'numpy.float64'>'
	with 1786336 stored elements in Compressed Sparse Row format>